# Classification using manually implemented Softmax Regression
___

# <font color = "green"> This was created for maths understanding</font>

## 1. Import all the required libraries

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
%matplotlib notebook

## 2. Create the versions info file

In [2]:
import sklearn
import matplotlib
with open('requirements.txt', 'wt') as f:
    np_vers = 'numpy version: ' + np.__version__ + '\n'
    pd_vers = 'pandas version: ' + pd.__version__ + '\n'
    sk_vers = 'sklearn version(mnist dataset only): ' + sklearn.__version__ + '\n'
    mat_vers = 'matplotlib version: ' + matplotlib.__version__ + '\n'
    f.write(np_vers)
    f.write(pd_vers)
    f.write(sk_vers)
    f.write(mat_vers)
    f.close()

## 3. Load the Iris dataset (for multiclassification)

In [3]:
Iris = load_iris()
properties = Iris['data']
labels = Iris['target']

In [4]:
properties[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [5]:
labels[:5]

array([0, 0, 0, 0, 0])

## 4. Use one hot vector encoding

In [6]:
from collections import Counter
def OneListEncoder(list_cls):
    '''
    Encodes an array of class instances using specific vectors (zeros and ones)
    '''
    cls = list(Counter(list_cls).keys())
    vector = np.zeros(len(cls), dtype = 'int')
    encoded = []
    for instance in list_cls:
        copy = np.copy(vector)
        index = cls.index(instance)
        copy[index] += 1
        encoded.append(copy)
    return encoded

In [7]:
labels = OneListEncoder(labels)

In [8]:
labels[:5]

[array([1, 0, 0]),
 array([1, 0, 0]),
 array([1, 0, 0]),
 array([1, 0, 0]),
 array([1, 0, 0])]

In [9]:
data = pd.DataFrame(data = properties, columns = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
data.insert(loc = len(data.columns), column = 'Labels', value = labels)
data

,Sepal Length,Sepal Width,Petal Length,Petal Width,Labels
0,5.1,3.5,1.4,0.2,"[1, 0, 0]"
1,4.9,3.0,1.4,0.2,"[1, 0, 0]"
2,4.7,3.2,1.3,0.2,"[1, 0, 0]"
3,4.6,3.1,1.5,0.2,"[1, 0, 0]"
4,5.0,3.6,1.4,0.2,"[1, 0, 0]"
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,"[0, 0, 1]"
146,6.3,2.5,5.0,1.9,"[0, 0, 1]"
147,6.5,3.0,5.2,2.0,"[0, 0, 1]"
148,6.2,3.4,5.4,2.3,"[0, 0, 1]"


## 5. Split the data into train and test sets

In [10]:
from math import floor
def trainTestSplit(data, test_ratio = 0.33):
    '''
    Splits the dataset into training and test data.
    The ratio of test data is 0.33 by default.
    '''
    np.random.seed(42)
    shuf_indices = np.random.permutation(len(data))
    test_length = floor(test_ratio * len(data))
    i_test = shuf_indices[:test_length]
    i_train = shuf_indices[test_length:]
    return data.iloc[i_train], data.iloc[i_test]

In [11]:
data_train, data_test = trainTestSplit(data, test_ratio = 0.2)

In [12]:
data_train.head(5)

,Sepal Length,Sepal Width,Petal Length,Petal Width,Labels
22,4.6,3.6,1.0,0.2,"[1, 0, 0]"
15,5.7,4.4,1.5,0.4,"[1, 0, 0]"
65,6.7,3.1,4.4,1.4,"[0, 1, 0]"
11,4.8,3.4,1.6,0.2,"[1, 0, 0]"
42,4.4,3.2,1.3,0.2,"[1, 0, 0]"


In [13]:
data_test.head(5)

,Sepal Length,Sepal Width,Petal Length,Petal Width,Labels
73,6.1,2.8,4.7,1.2,"[0, 1, 0]"
18,5.7,3.8,1.7,0.3,"[1, 0, 0]"
118,7.7,2.6,6.9,2.3,"[0, 0, 1]"
78,6.0,2.9,4.5,1.5,"[0, 1, 0]"
76,6.8,2.8,4.8,1.4,"[0, 1, 0]"


## 6. Create X and y datasets

In [14]:
X_train = data_train.drop('Labels', axis = 1)
y_train = data_train['Labels']
X_test = data_test.drop('Labels', axis = 1)
y_test = data_test['Labels']

In [15]:
X_train.head(5)

,Sepal Length,Sepal Width,Petal Length,Petal Width
22,4.6,3.6,1.0,0.2
15,5.7,4.4,1.5,0.4
65,6.7,3.1,4.4,1.4
11,4.8,3.4,1.6,0.2
42,4.4,3.2,1.3,0.2


In [16]:
y_train[:10]

22     [1, 0, 0]
15     [1, 0, 0]
65     [0, 1, 0]
11     [1, 0, 0]
42     [1, 0, 0]
146    [0, 0, 1]
51     [0, 1, 0]
27     [1, 0, 0]
4      [1, 0, 0]
32     [1, 0, 0]
Name: Labels, dtype: object

## 7. Scale the data

In [17]:
class DataScalerMean:
    mean = 1
    def __init__(self): 
        return
    
    def fit(self, data):
        self.mean = []
        for instance in data.columns:
            self.mean.append(np.mean(data[instance]))
        
    def transform(self, data):
        scaled_data = []
        for instance, idx in zip(data.columns, range(len(self.mean))):
            scaled_data.append(data[instance]/self.mean[idx])
        return pd.DataFrame(scaled_data).T
    
    def fit_transform(self, data):
        self.fit(data)
        scaled_data = self.transform(data)
        return scaled_data
    

In [18]:
scaler = DataScalerMean()
X_scaled_train = scaler.fit_transform(X_train)

In [19]:
X_scaled_train

,Sepal Length,Sepal Width,Petal Length,Petal Width
22,0.791852,1.175830,0.268336,0.169014
15,0.981208,1.437126,0.402504,0.338028
65,1.153350,1.012520,1.180680,1.183099
11,0.826280,1.110506,0.429338,0.169014
42,0.757424,1.045182,0.348837,0.169014
...,...,...,...,...
71,1.050065,0.914535,1.073345,1.098592
106,0.843494,0.816549,1.207513,1.436620
14,0.998422,1.306478,0.322004,0.169014
92,0.998422,0.849211,1.073345,1.014085


## 8. Create a SoftmaxRegression class, that uses BatchGD

**Softmax Score:**

$\large s(x^i) = \theta^T \cdot x^i$


**Softmax function:**

$\large \hat p_k = \frac {exp(\space s_k(\space x^i \space)\space)}{\sum _{j = 1} ^{K} exp(\space s_j (\space x^i \space) \space)}$

$\text{Where:} \\
K \text{ the is number of all clusters} \\
k \text{ is the current cluster's index} \\
\hat p_k \text{is the estimated probability of an instance belonging to a certain cluster}$

**Gradient:**

$\large \nabla _{\theta ^k} \space J(\Theta)  =  \frac{1}{m} \sum ^{m} _{i = 1}(\hat p ^{(i)} _k  - y ^{(i)} _k) x^{(i)}$

In [20]:
class SoftmaxRegression:
    theta = np.array([], dtype = 'float')
    
    def __init__(self, solver = 'BGD', n_iter = 100, learn_rate = 0.1):
        self.n_iter = n_iter
        self.learn_rate = learn_rate
        self.solver = solver
        
    def softmax_score(self, X_i, Theta_k): #softmax score realization
        return np.dot(X_i.T, Theta_k)
    
    def softmax_function(self, X_i, Theta, k):
        denom = 0
        for j in range(len(Theta)):
            denom += np.exp(self.softmax_score(X_i, Theta[j])) #get the denominator
        return (np.exp(self.softmax_score(X_i, Theta[k]))/denom) #probabililty of instance i being in k-th class
    
    def fit(self, X, y):
        '''
        Trains a Softmax Regression model given a pandas DataFrame X and list of one-hot-encoded labels y
        '''
        
        if(self.solver != 'BGD'): return "no such solver" #Check if solver is right
        
        #copy the data and add a column of ones for training
        X_0 = pd.DataFrame.copy(X)
        X_0.insert(loc = 0, column = 'x0', value = np.ones(len(X_0), dtype = 'int')) 
    
        len_X_0 = len(X_0)
        width_X_0 = len(X_0.columns) #get length and witdh of X_0 matrix
        
        len_theta = len(y[0])
        self.theta = np.random.randn(len_theta, width_X_0) #set random theta and get its length
        
        for iteration in range(self.n_iter): #in each iteration out of all
            gradient = []
            for v_idx in range(len_theta): #for each theta vector
                vector_der = np.zeros(width_X_0, dtype = 'float')
                for idx in range(len_X_0): #compute the derivative of the cost function
                    vector_der += (self.softmax_function(X_0.iloc[idx], self.theta, v_idx) - \
                                   y.iloc[idx][v_idx]) * X_0.iloc[idx]
                
                #and append it to the gradient
                gradient.append(np.multiply(1/len_X_0, vector_der))   
                
            #then use Gradient Descent's step
            self.theta = np.subtract(self.theta, np.multiply(self.learn_rate, np.array(gradient)))
        
    def predict(self, X):
        '''
        Given a DataFrame X returns the prediction of a trained model
        '''
        #copy the data and add a column of ones
        X_0 = pd.DataFrame.copy(X)
        X_0.insert(loc = 0, column = 'x0', value = np.ones(shape = len(X_0), dtype = 'int'))
        
        #get length of both theta and X_0
        len_theta = len(self.theta)
        len_X_0 = len(X_0)

        #create a matrix of probabilities - for each instance we will have n probabilities (n - number of clusters)
        self.proba = np.zeros(shape = (len_X_0, len_theta), dtype = 'float')
        
        for instance in range(len(X_0)): # for each instance
            for k in range(len_theta): #for each vector of Theta matrix
                #compute the probability of instance belonging to k-th cluster
                self.proba[instance][k] += self.softmax_function(X_0.iloc[instance], self.theta, k)
                
        #create an empty array for predictions
        prediction = np.array([], dtype = int)
        
        for instance in self.proba: #for each instance in probability matrix
            #append the array by the argument of the highest probability (thus, giving us a number)
            prediction = np.append(prediction, np.argmax(instance)) 
            
        return prediction

In [21]:
#Create the instance of SoftmaxRegression class with number of iterations equal to 100
SRg = SoftmaxRegression(n_iter = 500)

In [22]:
#Fit the scaled data
SRg.fit(X_train, y_train)

In [23]:
#get the predictions 
X_scaled_test = scaler.transform(X_test)
y_predict_labels = SRg.predict(X_test)
y_predict_labels

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 1, 2, 2, 2, 0, 0], dtype=int64)

In [24]:
#print test labels
y_test_labels = []
for instance in y_test:
    y_test_labels.append(np.argmax(instance))
print(y_test_labels)

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]


## 9. Evaluate the accuracy of a model

In [25]:
def eval_accuracy(y, y_pred):
    len_y = len(y)
    correct = 0
    for idx in range(len_y):
        if y[idx] == y_pred[idx]: correct += 1
    return correct/len_y

In [26]:
eval_accuracy(y_test_labels, y_predict_labels)

0.9666666666666667

In [27]:
#When predicting, the model calculates probabilities
(SRg.proba[0]*100).round(2)

array([ 0.77, 74.23, 25.  ])

In [28]:
SRg.proba[:5]

array([[7.66633986e-03, 7.42313509e-01, 2.50020151e-01],
       [9.77301993e-01, 2.26978466e-02, 1.60529688e-07],
       [5.49805063e-07, 1.34025147e-02, 9.86596935e-01],
       [1.01021534e-02, 7.10115226e-01, 2.79782620e-01],
       [5.91621558e-03, 8.48764355e-01, 1.45319430e-01]])